In [ ]:
import numpy as np
import scipy.optimize as opt
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
import matplotlib.colors
%matplotlib inline
import networkx as nx
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
import pandas as pd
from joblib import Parallel, delayed 
import random
from scipy.linalg import sqrtm
import scipy
from numpy import linalg as LA
from numpy.linalg import matrix_power, inv
from signet.cluster import Cluster
from signet.block_models import SSBM
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import statistics
import igraph as ig
import json
import time

from contextlib import closing
from dataclasses import dataclass
import os
import pathlib
import shutil
import subprocess
import tarfile
import urllib.request
import numpy as np
import pandas as pd
import shapefile

plt.style.use('dissertation.mplstyle')

# 5 Numerical Experiments

## 5.1 Implementation

### 5.1.1 Similarity matrices

In [ ]:
# Adjacency matrices

def A_u(A):
    return abs(A)

def A_plus(A): 
    return np.where(A > 0, A, 0)


def A_minus(A): 
    return abs(np.where(A < 0, A, 0))


# Diagonal matrix of degrees

def D(A):
    return np.diag(A_u(A).sum(axis=1))


def D_inv(A):
    return np.diag(1/(A_u(A).sum(axis=1)))


# Transition matrix blocks

def T_plus(A): 
    return D_inv(A) @ A_plus(A)


def T_minus(A):
    return D_inv(A) @ A_minus(A)


def T_positive(A):
    A_p = A_plus(A)
    return D_inv(A_p) @ A_p

In [ ]:
# Transition matrices

def T_strong(A):
    T_p = T_plus(A)
    T_m = T_minus(A)
    return np.block([[T_p, T_m], [T_m, T_p]])


def T_weak_without_teleport(A):
    n = np.shape(A)[0]
    T_p = T_plus(A)
    T_m = T_minus(A)
    return np.block([[T_p, T_m], [np.zeros((n,n)), T_p]])


def T_weak(A, i):
    n = np.shape(A)[0]
    T_missing_prob = T_weak_without_teleport(A)
    missing_prob = 1 - T_missing_prob[n:,:].sum(axis=1)
    T_missing_prob[n:,i] = missing_prob
    return T_missing_prob

In [ ]:
# Similarity matrices

# Returns the similarity matrix S_S(alpha)
def similarity_matrix_strong(A, alpha):
    n = np.shape(A)[0]
    return similarity_matrix(A, np.eye(n, 2*n), alpha, T_strong(A))


# Returns a tuple of the effective rate of continuation alpha' and
# the similarity matrix S_W(alpha)
def similarity_matrix_weak(A, alpha, tol=1e-10, max_steps=None):
    n = np.shape(A)[0]
    max_steps = n if max_steps is None else max_steps
    X0 = np.eye(n, 2*n)
    teleporting_rate = 0
    
    # With probability alpha, continue according to transition
    # matrix T_weak_without_teleport, and all other walkers teleport
    # back to starting node
    Xt = X0
    Xt_prev = X0
    for t in range(max_steps):
        Xt = alpha * Xt @ T_weak_without_teleport(A)
        missing_prob = 1 - Xt.sum(axis=1)
        Xt += X0 * missing_prob[:,np.newaxis]
        teleporting_rate += missing_prob.sum()

        if (np.linalg.norm(Xt - Xt_prev) < tol):
            break
        Xt_prev = Xt
        
    return (1 - teleporting_rate / n / t,
            Xt[:,:n] - Xt[:,n:])


# Returns the similarity matrix S_W(alpha)
# Not used, more efficient version above
def similarity_matrix_weak_inefficient(A, alpha):
    n = np.shape(A)[0]
    return np.concatenate([
        similarity_matrix(
            A,
            basis_vector(2*n, i)[np.newaxis,:], 
            alpha, 
            T_weak(A, i)
        ) for i in range(n)
    ])


# Helper function for returning a similarity matrix based on a 
# random walk with restart, with teleportation at constant rate 
# 1 - alpha
def similarity_matrix(A, X0, alpha, T):
    n = np.shape(A)[0]
    I = np.identity(2*n)
    sim_matrix_ext = (1 - alpha) * X0 @ inv(I - alpha * T)
    return sim_matrix_ext[:,:n] - sim_matrix_ext[:,n:]
        

def basis_vector(n, i):
    v = np.zeros(n)
    v[i] = 1
    return v

In [ ]:
# Turn positive and negative values into +/- 1
# (good for visualisation, and possibly clustering)

def matrix_filter(A, thresh=0):
    return np.where(A > thresh, 1.0, np.where(A < -thresh, -1.0, 0))

In [ ]:
class Function:
    def __init__(self, label, colour, marker, definition):
        self.label = label
        self.colour = colour
        self.marker = marker
        self.definition = definition
        

FUNCTIONS = {
    'Aspec': Function(r'spectral, $A$', 'black', '.', lambda Ap, An, k, smw, sms: (Cluster((Ap, An))).spectral_cluster_adjacency(k=k, normalisation='none', eigens=k-1)),
    'Amod': Function(r'modularity, $A$', 'grey', 'o', lambda Ap, An, k, smw, sms: traag_modularity_method((Ap - An).toarray(), k)), 
    'Lsym': Function(r'$\overline{L}_\mathrm{sym}$', 'tab:brown', 's', lambda Ap, An, k, smw, sms: (Cluster((Ap, An))).spectral_cluster_laplacian(k=k, normalisation='sym', eigens=k-1)),
    'SPONGE': Function('SPONGE', '#EE9AFC', (6,2,0), lambda Ap, An, k, smw, sms: (Cluster((Ap, An))).SPONGE(k=k, eigens=k-1)),
    'SPONGEsym': Function(r'SPONGE$_\mathrm{sym}$', '#A608C2', '*', lambda Ap, An, k, smw, sms: (Cluster((Ap, An))).SPONGE_sym(k=k, eigens=k-1)),
    'spectral': {
        'weak': {'scaled': Function('spectral, weak, scaled', '#94FF9B', '2', lambda Ap, An, k, smw, sms: spectral_method(D((Ap - An).toarray()) @ smw, k=k, eigens=k-1)), 
                 'unscaled': Function('spectral, weak, unscaled', '#19BA0D', '1', lambda Ap, An, k, smw, sms: spectral_method(smw, k=k, eigens=k-1)), 
                 'filtered': Function('spectral, weak, filtered', '#0c6306', '+', lambda Ap, An, k, smw, sms: spectral_method(matrix_filter(smw), k=k, eigens=k-1))},
        'strong': {'scaled': Function('spectral, strong, scaled', '#F78D8D', '^', lambda Ap, An, k, smw, sms: spectral_method(D((Ap - An).toarray()) @ sms, k=k, eigens=k-1)), 
                   'unscaled': Function('spectral, strong, unscaled', '#FF0000', 'v', lambda Ap, An, k, smw, sms: spectral_method(sms, k=k, eigens=k-1)), 
                   'filtered': Function('spectral, strong, filtered', '#850909', 'P', lambda Ap, An, k, smw, sms: spectral_method(matrix_filter(sms), k=k, eigens=k-1))},
    },
    'modularity': {
        'weak': {'scaled': Function('modularity, weak, scaled', '#94DFFF', '3', lambda Ap, An, k, smw, sms: traag_modularity_method(D((Ap - An).toarray()) @ smw, k)), 
                 'unscaled': Function('modularity, weak, unscaled', '#0A9FF5', '4', lambda Ap, An, k, smw, sms: traag_modularity_method(smw, k)), 
                 'filtered': Function('modularity, weak, filtered', '#0A1ED1', 'x', lambda Ap, An, k, smw, sms: traag_modularity_method(matrix_filter(smw), k))}, 
        'strong': {'scaled': Function('modularity, strong, scaled', '#FCD395', '<', lambda Ap, An, k, smw, sms: traag_modularity_method(D((Ap - An).toarray()) @ sms, k)), 
                   'unscaled': Function('modularity, strong, unscaled', '#F29305', '>', lambda Ap, An, k, smw, sms: traag_modularity_method(sms, k)), 
                   'filtered': Function('modularity, strong, filtered', '#C25604', 'X', lambda Ap, An, k, smw, sms: traag_modularity_method(matrix_filter(sms), k))}, 
    }
}

In [ ]:
# similarity_matrix_data format: [((x, (Ap, An), true_assignment), [ari per function])]
def get_ari_data_point(functions, similarity_matrix_data):
    x, Ap, An, true_assignment, k, sm_weak, sm_strong = similarity_matrix_data

    aris = []
    for function in functions:
        prediction = function(Ap, An, k, sm_weak, sm_strong)
        aris.append(adjusted_rand_score(prediction, true_assignment))

    return (x, (Ap, An), true_assignment), aris

In [ ]:
# Save, load and combine ARI data
# data format: [((x, (Ap, An), true_assignment), [ari per function])]
# x could be eta or alpha (x-axis variable)

def combine_ari_data(data1, data2):
    n1, a1 = zip(*data1)
    n2, a2 = zip(*data2)
    return list(zip(n1, list(map(lambda l1, l2: l1 + l2, a1, a2))))


def save_ari_data(data, prefix=''):
    data_without_sparse_matrices = []
    for i, (network, aris) in enumerate(data):
        eta, (Ap, An), true_assignment = network
        data_without_sparse_matrices.append(((eta, list(map(int, true_assignment))), aris))
        scipy.sparse.save_npz(f'{prefix}Ap{i}.npz', Ap)
        scipy.sparse.save_npz(f'{prefix}An{i}.npz', An)
    with open(f'{prefix}data.json', 'w') as f:
        json.dump(data_without_sparse_matrices, f)
        
        
def load_ari_data(prefix=''):
    with open(f'{prefix}data.json', 'r') as f:
        data_without_sparse_matrices = json.load(f)
        
    data = []
    for i, ((eta, true_assignment), aris) in enumerate(data_without_sparse_matrices):
        Ap = scipy.sparse.load_npz(f'{prefix}Ap{i}.npz')
        An = scipy.sparse.load_npz(f'{prefix}An{i}.npz')
        data.append(((eta, (Ap, An), true_assignment), aris))
    return data

### 5.1.2 Community detection

In [ ]:
# Traag modularity method for community detection
def traag_modularity_method(A, k):
    g = ig.Graph.from_networkx(nx.from_numpy_array(A))
    predictions = g.community_spinglass(weights='weight', spins=k, implementation='neg')
    return predictions.membership


# Spectral method for community detection
def spectral_method(A, k, eigens=None):
    eigens = eigens if eigens else k
    B = (A + A.T) / 2
    
    w, v = np.linalg.eigh(B)
    x = KMeans(n_clusters=k).fit(v[:,-eigens:])
    return x.labels_

## 5.2 Synthetic networks

In [ ]:
MODULARITY_FUNCTIONS = [
    FUNCTIONS['modularity']['weak']['scaled'],
    FUNCTIONS['modularity']['weak']['unscaled'],
    FUNCTIONS['modularity']['weak']['filtered'],
    FUNCTIONS['modularity']['strong']['scaled'],
    FUNCTIONS['modularity']['strong']['unscaled'],
    FUNCTIONS['modularity']['strong']['filtered'],
]

SPECTRAL_FUNCTIONS = [
    FUNCTIONS['spectral']['weak']['scaled'],
    FUNCTIONS['spectral']['weak']['unscaled'],
    FUNCTIONS['spectral']['weak']['filtered'],
    FUNCTIONS['spectral']['strong']['scaled'],
    FUNCTIONS['spectral']['strong']['unscaled'],
    FUNCTIONS['spectral']['strong']['filtered'],
]

In [ ]:
def SSBM_asym(n, k, probs, etas):    
    nodes_per_cluster = n // k
    cluster_sizes = (nodes_per_cluster * np.ones(k)).astype(int)
    G = nx.stochastic_block_model(cluster_sizes, probs)
    
    co = np.repeat(np.arange(k), nodes_per_cluster)
    
    for u,v in G.edges():
        eta = etas[co[u]][co[v]]
        flip_sign = np.random.binomial(1, eta)
        if co[u] == co[v]:
            G.edges[u,v]['weight'] = -1 if flip_sign else 1
        else:
            G.edges[u,v]['weight'] = 1 if flip_sign else -1
            
    A = nx.to_numpy_array(G)
    return (scipy.sparse.csc_matrix(A_plus(A)), 
            scipy.sparse.csc_matrix(A_minus(A))), co

In [ ]:
def sroc_probs(n, k, 
               p_within, p_between, p_other, 
               eta_within, eta_between, eta_other):
    probs = np.zeros((k,k))
    etas = np.zeros((k,k))
    for i in range(k):
        for j in range(i,k):
            if i == j:
                probs[i,j] = p_within
                etas[i,j] = eta_within
            elif j - i == 1 or i == 0 and j == k-1:
                probs[i,j] = p_between
                probs[j,i] = p_between
                etas[i,j] = eta_between
                etas[j,i] = eta_between
            else:
                probs[i,j] = p_other
                probs[j,i] = p_other
                etas[i,j] = eta_other
                etas[j,i] = eta_other
                
    return probs, etas

In [ ]:
def plot_ari_data(xs, results, 
                  colours, markers, labels, 
                  ax, title, xlabel='', ylabel='', 
                  legend_on=True, legend_order=None, legend_markerfirst=False, filename=''):
    networks, aris = zip(*results)
    for i, func_aris in enumerate(zip(*aris)):
        aris = []
        errors = []
        for x_aris in np.reshape(func_aris, (-1,repetitions)):
            aris.append(statistics.mean(x_aris))
            errors.append(statistics.stdev(x_aris))
        ax.errorbar(xs, aris, errors, 
                    color=colours[i], 
                    marker=markers[i], 
                    label=labels[i],
                    linewidth=0.8,
                    markersize=3,
                   )
        
    ax.set_xlim([xs[0], xs[-1]])
    ax.set_ylim([-0.02,1.02])
    ax.set_title(title, fontsize=11, x=0.93, y=0.95, pad=-10)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if legend_on:
        handles, labels = ax.get_legend_handles_labels()
        if legend_order is None:
            legend_order = np.arange(len(handles))
        ax.legend([handles[idx] for idx in legend_order],[labels[idx] for idx in legend_order],
                  frameon=False, fontsize=6, markerfirst=legend_markerfirst)
    if filename != '':
        plt.savefig(filename, bbox_inches='tight')

### 5.2.1 Comparison of the random walk similarity matrices

In [ ]:
def get_ari_data_varying_noise(functions, n, k, p, alpha, repetitions, etas, 
                               build_similarity_matrices, networks=None):
    similarity_matrices = Parallel(n_jobs=-1, verbose=10)(
        delayed(build_similarity_matrices)(
            n, k, p, alpha, eta, None if networks is None else networks[repetitions*i + j]
        ) for i, eta in enumerate(etas) for j, _ in enumerate(range(repetitions))
    )
    return Parallel(n_jobs=-1, verbose=10)(
        delayed(get_ari_data_point)(
            functions, similarity_matrix_data
        ) for similarity_matrix_data in similarity_matrices
    )


def get_ari_data_varying_alpha(functions, n, k, p, eta, repetitions, alphas, 
                               build_similarity_matrices, networks=None):
    similarity_matrices = Parallel(n_jobs=-1, verbose=10)(
        delayed(build_similarity_matrices)(
            n, k, p, alpha, eta, None if networks is None else networks[repetitions*i + j]
        ) for i, alpha in enumerate(alphas) for j, _ in enumerate(range(repetitions))
    )
    return Parallel(n_jobs=-1, verbose=10)(
        delayed(get_ari_data_point)(
            functions, similarity_matrix_data
        ) for similarity_matrix_data in similarity_matrices
    )

#### Varying noise on the SSBM

In [ ]:
def plot_ari_data_ssbm(functions, colours, markers, labels,
                       n, k, p, alpha, repetitions,
                       min_eta, max_eta, eta_inc, networks=None,
                       legend_on=True, filename=''):
    
    etas = np.arange(min_eta, max_eta, eta_inc)
    results = get_ari_data_varying_noise(functions, n, k, p, alpha, repetitions, etas, 
                                         build_similarity_matrices_ssbm, networks)
    fig, ax = plt.subplots()
    plot_ari_data(etas, results, colours, markers, labels, 
                  ax, '', legend_on, filename)
    return results

    
def build_similarity_matrices_ssbm(n, k, p, alpha, eta, network=None):
    if network is None:
        (Ap, An), true_assignment = SSBM(n=n, k=k, pin=p, etain=eta)
    else:
        (_, (Ap, An), true_assignment), aris = network
    A = (Ap - An).toarray()
    alpha_s, sm_weak = similarity_matrix_weak(A, alpha)
    sm_strong = similarity_matrix_strong(A, alpha_s)
    return eta, Ap, An, true_assignment, k, sm_weak, sm_strong

In [ ]:
# Constants

community_size = 50
p = 0.2
alpha = 0.85
repetitions = 5
min_eta = 0.0
max_eta = 0.401
eta_inc = 0.02

In [ ]:
# Generate the data for each subfigure of Figure 5.6

k = 2
n = community_size * k
PLOT_FUNCTIONS = SPECTRAL_FUNCTIONS

data_ssbm_2_spectral = plot_ari_data_ssbm(
    [f.definition for f in PLOT_FUNCTIONS],
    [f.colour for f in PLOT_FUNCTIONS],
    [f.marker for f in PLOT_FUNCTIONS],
    [f.label for f in PLOT_FUNCTIONS],
    n, k, p, alpha, repetitions, min_eta, max_eta, eta_inc,
    legend_on=True, filename=''
)
save_ari_data(data_ssbm_2_spectral, 'ssbm_2_spectral-')

In [ ]:
# Figure 5.6

XLABELS = [['', ''], ['', ''], ['Modularity optimisation methods', 'Spectral clustering methods']]
YLABELS = ['$k = 2$', '$k = 5$', '$k = 10$']
DATA_TO_PLOT = [data_ssbm_2_traag, data_ssbm_2_spectral, 
                data_ssbm_5_traag, data_ssbm_5_spectral, 
                data_ssbm_10_traag, data_ssbm_10_spectral]

fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, layout='constrained')
for i in range(3):
    plot_ari_data(np.arange(min_eta, max_eta, eta_inc), DATA_TO_PLOT[2*i + 0], 
                  [f.colour for f in MODULARITY_FUNCTIONS],
                  [f.marker for f in MODULARITY_FUNCTIONS],
                  [f.label for f in MODULARITY_FUNCTIONS],
                  axs[i, 0], title=f'({chr(ord("a") + 2*i + 0)})', xlabel=XLABELS[i][0], ylabel=YLABELS[i], legend_on=(i==0), filename='')
    
    plot_ari_data(np.arange(min_eta, max_eta, eta_inc), DATA_TO_PLOT[2*i + 1], 
                  [f.colour for f in SPECTRAL_FUNCTIONS],
                  [f.marker for f in SPECTRAL_FUNCTIONS],
                  [f.label for f in SPECTRAL_FUNCTIONS],
                  axs[i, 1], title=f'({chr(ord("a") + 2*i + 1)})', xlabel=XLABELS[i][1], ylabel='', legend_on=(i==0), filename='')
fig.supxlabel(r'Sign flip probability ($\eta$)', fontsize=11)
fig.supylabel('Adjusted Rand index', fontsize=11)
fig.subplots_adjust(hspace=0.1, wspace=0.1)
plt.savefig('ssbm.pdf', bbox_inches='tight')

In [ ]:
# Figure 5.7

k = 10
n = community_size * k
eta = 0.2

(Ap, An), true_assignment = SSBM(n = n, k = k, pin = p, etain = eta)
A = (Ap - An).toarray()
degs = A_u(A).sum(axis=1)

plt.figure(figsize=(2.2,1.65))
plt.hist(degs, bins=np.arange(degs.min(), degs.max()+1), color='grey')
plt.xlim([60,140])
plt.ylim([0,40])
plt.xlabel('Degree')
plt.ylabel('Number of nodes')
plt.savefig('degree-distribution.pdf', bbox_inches='tight')

#### Varying noise on the SROC

In [ ]:
def plot_ari_data_sroc(functions, colours, markers, labels,
                       n, k, p_within, p_between, alpha, repetitions,
                       min_eta, max_eta, eta_inc, networks=None,
                       legend_on=True, filename=''):
    
    etas = np.arange(min_eta, max_eta, eta_inc)
    results = get_ari_data_varying_noise(functions, n, k, (p_within, p_between), alpha, repetitions, etas, 
                                         build_similarity_matrices_sroc, networks)
    fig, ax = plt.subplots()
    plot_ari_data(etas, results, colours, markers, labels, 
                  ax, '', legend_on, filename)
    return results

    
def build_similarity_matrices_sroc(n, k, ps, alpha, eta, network=None):
    p_within, p_between = ps
    probs, etas = sroc_probs(n, k, p_within, p_between, 0, eta, 0, 0)
    
    if network is None:
        (Ap, An), true_assignment = SSBM_asym(n, k, probs, etas)
    else:
        (_, (Ap, An), true_assignment), aris = network
    A = (Ap - An).toarray()
    alpha_s, sm_weak = similarity_matrix_weak(A, alpha)
    sm_strong = similarity_matrix_strong(A, alpha_s)
    return alpha, Ap, An, true_assignment, k, sm_weak, sm_strong

In [ ]:
# Constants

community_size = 50
alpha = 0.85
p_within = 0.9
p_between = 0.02
repetitions = 5
min_eta = 0.0
max_eta = 0.501
eta_inc = 0.02

In [ ]:
# Generate the data for each subfigure of Figure 5.8

k = 2
n = community_size * k
PLOT_FUNCTIONS = SPECTRAL_FUNCTIONS

data_sroc_2_spectral = plot_ari_data_sroc(
    [f.definition for f in PLOT_FUNCTIONS],
    [f.colour for f in PLOT_FUNCTIONS],
    [f.marker for f in PLOT_FUNCTIONS],
    [f.label for f in PLOT_FUNCTIONS],
    n, k, p_within, p_between, alpha, repetitions, min_eta, max_eta, eta_inc,
    legend_on=True, filename=''
)
save_ari_data(data_sroc_2_spectral, 'sroc_2_spectral-')

In [ ]:
# Figure 5.8

XLABELS = [['', ''], ['', ''], ['Modularity optimisation methods', 'Spectral clustering methods']]
YLABELS = ['$k = 2$', '$k = 5$', '$k = 10$']
DATA_TO_PLOT = [data_sroc_2_traag, data_sroc_2_spectral, 
                data_sroc_5_traag, data_sroc_5_spectral, 
                data_sroc_10_traag, data_sroc_10_spectral]

fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, layout='constrained')
for i in range(3):
    plot_ari_data(np.arange(min_eta, max_eta, eta_inc), DATA_TO_PLOT[2*i + 0], 
                  [f.colour for f in MODULARITY_FUNCTIONS],
                  [f.marker for f in MODULARITY_FUNCTIONS],
                  [f.label for f in MODULARITY_FUNCTIONS],
                  axs[i, 0], title=f'({chr(ord("a") + 2*i + 0)})', xlabel=XLABELS[i][0], ylabel=YLABELS[i], legend_on=(i==0), filename='')
    
    plot_ari_data(np.arange(min_eta, max_eta, eta_inc), DATA_TO_PLOT[2*i + 1], 
                  [f.colour for f in SPECTRAL_FUNCTIONS],
                  [f.marker for f in SPECTRAL_FUNCTIONS],
                  [f.label for f in SPECTRAL_FUNCTIONS],
                  axs[i, 1], title=f'({chr(ord("a") + 2*i + 1)})', xlabel=XLABELS[i][1], ylabel='', legend_on=(i==0), filename='')
fig.supxlabel(r'Sign flip probability ($\eta$)', fontsize=11)
fig.supylabel('Adjusted Rand index', fontsize=11)
fig.subplots_adjust(hspace=0.1, wspace=0.1)
plt.savefig('sroc.pdf', bbox_inches='tight')

In [ ]:
# Generate similarity matrices for Figures 5.9 and 5.10
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","white","blue"])

k = 2
n = community_size * k
eta = 0.4

probs, etas = sroc_probs(n, k, p_within, p_between, 0, eta, 0, 0)
(Ap, An), true_assignment = SSBM_asym(n, k, probs, etas)
A = (Ap - An).toarray()
alpha_s, sm_weak = similarity_matrix_weak(A, alpha)
sm_strong = similarity_matrix_strong(A, alpha_s)

In [ ]:
pred = traag_modularity_method(matrix_filter(sm_weak), k)

In [ ]:
norm=plt.Normalize(vmax=-np.min(sm_weak), clip=True)
plt.imshow(sm_weak, cmap=cmap, norm=norm) # or matrix_filter(sm_weak) 
                                          # or matrix_filter(sm_weak)[np.argsort(pred), :][:,np.argsort(pred)]
                                          # or equivalent matrices for strong walk
plt.colorbar()
plt.axis('off')
plt.savefig(f'ssbm_k2_weak_unfiltered.pdf', bbox_inches='tight')
plt.show()

#### Varying the rate of teleportation

In [ ]:
def plot_ari_data_alpha(functions, colours, markers, labels,
                        n, k, p, eta, repetitions,
                        min_alpha, max_alpha, alpha_inc, networks=None,
                        legend_on=True, filename=''):
    
    alphas = np.arange(min_alpha, max_alpha, alpha_inc)
    results = get_ari_data_varying_alpha(functions, n, k, p, eta, repetitions, alphas, 
                                         build_similarity_matrices_ssbm, networks)
    fig, ax = plt.subplots()
    plot_ari_data(alphas, results, colours, markers, labels, 
                  ax, '', legend_on, filename)
    return results

In [ ]:
# Constants

community_size = 50
p = 0.2
repetitions = 5
min_alpha = 0.4
max_alpha = 1.001
alpha_inc = 0.02

In [ ]:
# Generate the data for each subfigure of Figure 5.11

k = 2
n = community_size * k
eta = 0.3
PLOT_FUNCTIONS = SPECTRAL_FUNCTIONS

data_alpha_ssbm_2_spectral = plot_ari_data_alpha(
    [f.definition for f in PLOT_FUNCTIONS],
    [f.colour for f in PLOT_FUNCTIONS],
    [f.marker for f in PLOT_FUNCTIONS],
    [f.label for f in PLOT_FUNCTIONS],
    n, k, p, eta, repetitions, min_alpha, max_alpha, alpha_inc,
    legend_on=True, filename='alpha_ssbm_2.pdf'
)
save_ari_data(data_alpha_ssbm_2_spectral, 'alpha_ssbm_2_spectral-')

In [ ]:
# Figure 5.11

XLABELS = [['', ''], ['', ''], ['Modularity optimisation methods', 'Spectral clustering methods']]
YLABELS = ['$k = 2, \eta = 0.3$', '$k = 5, \eta = 0.25$', '$k = 10, \eta = 0.15$']
DATA_TO_PLOT = [data_alpha_ssbm_2_traag, data_alpha_ssbm_2_spectral, 
                data_alpha_ssbm_5_traag, data_alpha_ssbm_5_spectral, 
                data_alpha_ssbm_10_traag, data_alpha_ssbm_10_spectral]

fig, axs = plt.subplots(3, 2, sharex=True, sharey=True, layout='constrained')
for i in range(3):
    plot_ari_data(np.arange(min_alpha, max_alpha, alpha_inc), DATA_TO_PLOT[2*i + 0], 
                  [f.colour for f in MODULARITY_FUNCTIONS],
                  [f.marker for f in MODULARITY_FUNCTIONS],
                  [f.label for f in MODULARITY_FUNCTIONS],
                  axs[i, 0], title=f'({chr(ord("a") + 2*i + 0)})', xlabel=XLABELS[i][0], ylabel=YLABELS[i], 
                  legend_on=False, filename='')
    
    plot_ari_data(np.arange(min_alpha, max_alpha, alpha_inc), DATA_TO_PLOT[2*i + 1], 
                  [f.colour for f in FUNCTIONS_TO_PLOT],
                  [f.marker for f in FUNCTIONS_TO_PLOT],
                  [f.label for f in FUNCTIONS_TO_PLOT],
                  axs[i, 1], title=f'({chr(ord("a") + 2*i + 1)})', xlabel=XLABELS[i][1], ylabel='', 
                  legend_on=False, filename='')
fig.subplots_adjust(hspace=0.1, wspace=0.1)
handles_traag, labels_traag = axs[0,0].get_legend_handles_labels()
handles_spectral, labels_spectral = axs[0,1].get_legend_handles_labels()
leg = fig.legend(handles_traag + handles_spectral,labels_traag + labels_spectral,
           frameon=True, fontsize=6,
           bbox_to_anchor=(0.125, 0.90, 0.775, 0.90), loc="lower left",
           mode="expand", borderaxespad=0, ncol=4
          )
leg.get_frame().set_edgecolor('k')
fig.supxlabel(r'Continuation rate ($\alpha$)', fontsize=11)
fig.supylabel('Adjusted Rand index', fontsize=11)
plt.savefig('ssbm_alpha.pdf', bbox_inches='tight')

### 5.2.2 Comparison to existing clustering algorithms

In [ ]:
COMPARE_FUNCTIONS = [
    FUNCTIONS['modularity']['weak']['scaled'],
    FUNCTIONS['modularity']['weak']['filtered'],
    FUNCTIONS['modularity']['strong']['scaled'],
    FUNCTIONS['spectral']['weak']['unscaled'],
    FUNCTIONS['spectral']['weak']['filtered'],
    FUNCTIONS['spectral']['strong']['unscaled'],
    FUNCTIONS['Amod'],
    FUNCTIONS['Aspec'],
    FUNCTIONS['Lsym'],
    FUNCTIONS['SPONGE'],
    FUNCTIONS['SPONGEsym']
]

In [ ]:
# Generate the data for each subfigure of Figure 5.12

k = 2
n = community_size * k
PLOT_FUNCTIONS = COMPARE_FUNCTIONS[6:]

data_ssbm_2_compare = get_and_plot_ari_data(
    [f.definition for f in PLOT_FUNCTIONS],
    [f.colour for f in PLOT_FUNCTIONS],
    [f.marker for f in PLOT_FUNCTIONS],
    [f.label for f in PLOT_FUNCTIONS],
    n, k, p, alpha, repetitions, min_eta, max_eta, eta_inc, networks=data_roc_2_traag,
    legend_on=True, filename='ssbm_2_compare.pdf'
)
save_ari_data(data_ssbm_2_compare, 'ssbm_2_compare-')

In [ ]:
# Figure 5.12

data_to_plot2 = combine_ari_data(
    data_ssbm_2_compare,
    combine_ari_data(
        select_ari_data(data_ssbm_2_traag, [0, 2, 3]),
        select_ari_data(data_ssbm_2_spectral2, [1, 2, 4]),
    )
)
data_to_plot5 = combine_ari_data(
    data_ssbm_5_compare,
    combine_ari_data(
        select_ari_data(data_ssbm_5_traag, [0, 2, 3]),
        select_ari_data(data_ssbm_5_spectral2, [1, 2, 4]),
    )
)
data_to_plot10 = combine_ari_data(
    data_ssbm_10_compare,
    combine_ari_data(
        select_ari_data(data_ssbm_10_traag, [0, 2, 3]),
        select_ari_data(data_ssbm_10_spectral2, [1, 2, 4]),
    )
)

data_to_plot2_roc = combine_ari_data(
    data_roc_2_compare,
    combine_ari_data(
        select_ari_data(data_roc_2_traag, [0, 2, 3]),
        select_ari_data(data_roc_2_spectral2, [1, 2, 4]),
    )
)
data_to_plot5_roc = combine_ari_data(
    data_roc_5_compare,
    combine_ari_data(
        select_ari_data(data_roc_5_traag, [0, 2, 3]),
        select_ari_data(data_roc_5_spectral2, [1, 2, 4]),
    )
)
data_to_plot10_roc = combine_ari_data(
    data_roc_10_compare,
    combine_ari_data(
        select_ari_data(data_roc_10_traag, [0, 2, 3]),
        select_ari_data(data_roc_10_spectral2, [1, 2, 4]),
    )
)

PLOT_FUNCTIONS = COMPARE_FUNCTIONS[6:] + COMPARE_FUNCTIONS[:6]

XLABELS = [['', ''], ['', ''], ['Evenly distributed SSBM', 'Ring of cliques']]
YLABELS = ['$k = 2$', '$k = 5$', '$k = 10$']
DATA_TO_PLOT = [data_to_plot2, data_to_plot2_roc, 
                data_to_plot5, data_to_plot5_roc, 
                data_to_plot10, data_to_plot10_roc]

fig, axs = plt.subplots(3, 2, sharex='col', sharey=True, layout='constrained')
for i in range(3):
    plot_ari_data(np.arange(0, 0.41, 0.02), DATA_TO_PLOT[2*i + 0], 
                  [f.colour for f in PLOT_FUNCTIONS],
                  [f.marker for f in PLOT_FUNCTIONS],
                  [f.label for f in PLOT_FUNCTIONS],
                  axs[i, 0], title=f'({chr(ord("a") + 2*i + 0)})', xlabel=XLABELS[i][0], ylabel=YLABELS[i], 
                  legend_on=(i==0), legend_order=list(range(5,11)) + list(range(5)), filename='')
    
    plot_ari_data(np.arange(0, 0.51, 0.02), DATA_TO_PLOT[2*i + 1], 
                  [f.colour for f in PLOT_FUNCTIONS],
                  [f.marker for f in PLOT_FUNCTIONS],
                  [f.label for f in PLOT_FUNCTIONS],
                  axs[i, 1], title=f'({chr(ord("a") + 2*i + 1)})', xlabel=XLABELS[i][1], ylabel='', 
                  legend_on=False, filename='')
fig.supxlabel(r'Sign flip probability ($\eta$)', fontsize=11)
fig.supylabel('Adjusted Rand index', fontsize=11)
fig.subplots_adjust(hspace=0.1, wspace=0.1)
plt.savefig('ssbm1compare.pdf', bbox_inches='tight')

In [ ]:
# Figure 5.13

PLOT_FUNCTIONS = COMPARE_FUNCTIONS[6:] + COMPARE_FUNCTIONS[:6]

data_to_plot = combine_ari_data(
    data_ssbm_20_compare,
    combine_ari_data(
        select_ari_data(data_ssbm_20_traag, [0, 2, 3]),
        select_ari_data(data_ssbm_20_spectral, [1, 2, 4]),
    )
)

fig, ax = plt.subplots(figsize=(3.2,2.4))
plot_ari_data(np.arange(0, 0.25, 0.02), data_to_plot, 
              [f.colour for f in PLOT_FUNCTIONS],
              [f.marker for f in PLOT_FUNCTIONS],
              [f.label for f in PLOT_FUNCTIONS],
              ax, xlabel=r'Sign flip probability ($\eta$)', ylabel='Adjusted Rand index', 
              title='', legend_on=True, legend_order=list(range(5,11)) + list(range(5)), legend_markerfirst=True, filename='')
    
plt.savefig('ssbm_20_compare.pdf', bbox_inches='tight')

## 5.3 Empirical networks

### 5.3.1 Sampson's monastery

In [ ]:
ALL_FUNCTIONS = MODULARITY_FUNCTIONS + SPECTRAL_FUNCTIONS + COMPARE_FUNCTIONS[6:]

In [ ]:
MARKER_TO_SIZE = {'s': 1.5, 
                  '.': 2, 'o': 2, 'P': 2,
                  '<': 3, '>': 3, '^': 3, 'v': 3, 'x': 3, 'X': 3, '*': 3, (6,2,0): 3, 
                  '1': 5, '2': 5, '3': 5, '4': 5, '+': 5} 

In [ ]:
G = nx.read_pajek("Sampson.paj")
A = nx.to_numpy_array(G)
A = A + A.T
co = np.asarray([1, 1, 3, 2, 2, 2, 1, 4, 2, 4, 2, 1, 4, 1, 1, 1, 3, 3]) - 1

In [ ]:
repetitions = 5
Ap = scipy.sparse.csc_matrix(A_plus(A))
An = scipy.sparse.csc_matrix(A_minus(A))
k = 4
alpha = 0.85
PLOT_FUNCTIONS = ALL_FUNCTIONS

alpha_s, smw = similarity_matrix_weak(A, alpha)
sms = similarity_matrix_strong(A, alpha_s)


aris = []
errors = []
for function in ALL_FUNCTIONS:
    func_aris = []
    for _ in range(repetitions):
        prediction = function.definition(Ap, An, k, smw, sms)
        func_aris.append(adjusted_rand_score(prediction, co))
    aris.append(statistics.mean(func_aris))
    errors.append(statistics.stdev(func_aris))

In [ ]:
plt.figure(figsize=(2.5,3.5))
plt.xlim([-0.25,1])
plt.xlabel('Adjusted Rand index')

for i, f in enumerate(PLOT_FUNCTIONS[::-1]):
    plt.scatter(aris[-1-i], f.label, c=f.colour, marker=f.marker, 
                s=5*MARKER_TO_SIZE[f.marker], linewidth=0.8)

plt.errorbar(
    aris[::-1], [f.label for f in PLOT_FUNCTIONS[::-1]], 
    xerr=errors[::-1], fmt='none',
    ecolor=[f.colour for f in PLOT_FUNCTIONS[::-1]],
    elinewidth=0.8
)
plt.yticks(fontsize=8)

plt.savefig('sampsons-monastery.pdf', bbox_inches='tight')

### 5.3.2 Australian rainfall

In [ ]:
RAINFALL_DATA_DIR = pathlib.Path('rainfall_data')
RAINFALL_DATA_SOURCE = 'ftp://ftp.bom.gov.au/anon/home/ncc/www/change/HQmonthlyR/HQ_monthly_prcp_txt.tar'

In [ ]:
@dataclass
class Station:
    id: str
    lat: float
    lon: float
    elev: float
    name: str

    @classmethod
    def from_station_line(cls, line: str):
        words = line.strip().split()
        if not words:
            return None
        return Station(id=words[0], lat=float(words[1]), lon=float(words[2]), elev=float(words[3]), name=' '.join(words[4:]))

    def data_file_path(self, freq = 'month'):
        return RAINFALL_DATA_DIR / f'prcphq.{self.id}.{freq}.txt'

    def load_data_series(self, freq = 'month'):
        with open(self.data_file_path(freq)) as data_file:
            data_lines = [line for line in data_file.read().splitlines() if line]
        if len(data_lines) < 2:
            return pd.DataFrame(columns=('rainfall',), dtype=float)

        missing_value = '999999.9'
        header_fields = data_lines[0].split()
        for field in header_fields:
            if field.startswith('missing_value='):
                missing_value = field.replace('missing_value=', '', 1)

        raw_data = dict()
        for line in data_lines[1:]:
            terms = line.split()
            if len(terms) < 3:
                continue
            if terms[2] == missing_value:
                continue
            else:
                raw_data[terms[0]] = float(terms[2])
        return pd.DataFrame.from_dict(raw_data, orient='index', dtype=float, columns=('rainfall',))

In [ ]:
# Download data
os.makedirs(RAINFALL_DATA_DIR, exist_ok=True)

print("Downloading", RAINFALL_DATA_SOURCE)
urllib.request.urlretrieve(RAINFALL_DATA_SOURCE, RAINFALL_DATA_DIR / 'source.tar')

print("Extracting", RAINFALL_DATA_DIR / 'source.tar')
with tarfile.open(RAINFALL_DATA_DIR / 'source.tar') as source_tar:
    source_tar.extractall(RAINFALL_DATA_DIR)

z_files = list(RAINFALL_DATA_DIR.glob('*.Z'))
print("Decompressing", len(z_files), "z-compressed files")
for z_file in z_files:
    subprocess.run(['uncompress', z_file.name], cwd=RAINFALL_DATA_DIR)

print("Done")

In [ ]:
with open(RAINFALL_DATA_DIR / 'HQMR_stations.txt') as sites_file:
    stations = [
        Station.from_station_line(line)
        for line in sites_file.read().splitlines()
        if line
    ]

In [ ]:
all_rainfall_data = pd.concat([station.load_data_series() for station in stations], axis='columns')

In [ ]:
A = all_rainfall_data.corr().to_numpy()

#### Visualisation

In [ ]:
AUSTRALIA_BOUNDARY = 'https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/AUS_2021_AUST_SHP_GDA2020.zip'

In [ ]:
# Download Australia boundary
os.makedirs(RAINFALL_DATA_DIR, exist_ok=True)

urllib.request.urlretrieve(AUSTRALIA_BOUNDARY, RAINFALL_DATA_DIR / 'australia.shp.zip')

In [ ]:
with shapefile.Reader(RAINFALL_DATA_DIR / 'australia.shp.zip') as australia_reader:
    def poly_area(poly):
        shifted = np.roll(poly, -1)
        return 0.5 * np.sum(poly[:,0] * shifted[:,1] - poly[:,1] * shifted[:,0])

    australia_boundaries = [
        pd.DataFrame(data=island, columns=('lon', 'lat'))
        for island in np.split(np.array(australia_reader.shape(0).points), australia_reader.shape(0).parts)
    ]
    australia_boundaries = sorted(australia_boundaries, key=lambda island: -1 * poly_area(island.to_numpy()))

In [ ]:
def plot_australia():
    for boundary in australia_boundaries[:10]: # Only draw the 10 largest boundaries
        plt.plot(boundary.to_numpy()[:,0], boundary.to_numpy()[:,1], color='black', linewidth=0.5)
    plt.xticks([])
    plt.yticks([]) 

def plot_station_group(group, marker='1', colour='b'):
    indexes = set(group)
    
    points = np.array([(st.lon, st.lat) for idx, st in enumerate(stations) if idx in indexes])
    plt.scatter(points[:,0], points[:,1], marker=marker, s=10*MARKER_TO_SIZE[marker], c=colour)

In [ ]:
norm = norm=plt.Normalize(vmin=-1)
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ["red","white","blue"])

In [ ]:
MARKER_TO_SIZE = {'s': 1.5, 
                  '.': 2, 'o': 2, 'P': 2,
                  '<': 3, '>': 3, '^': 3, 'v': 3, 'x': 3, 'X': 3, '*': 3, (6,2,0): 3, 
                  '1': 5, '2': 5, '3': 5, '4': 5, '+': 5}
MARKERS = ['s', 'o', '^', 'v', '<', '>', '2', '1', '3', '4']
COLOURS = ['#FF0000', '#F29305', '#19BA0D', '#0c6306', '#0A1ED1', '#0A9FF5', '#EE9AFC', '#A608C2', 'black', 'grey']

In [ ]:
Ap = scipy.sparse.csc_matrix(A_plus(A))
An = scipy.sparse.csc_matrix(A_minus(A))

alpha_s, smw = similarity_matrix_weak(A, alpha)
sms = similarity_matrix_strong(A, alpha_s)

In [ ]:
k = 6

preds = []
for i, f in enumerate(COMPARE_FUNCTIONS):
    pred = f.definition(Ap, An, k, smw, sms)
    preds.append(pred)

    plt.imshow(A[np.argsort(pred), :][:,np.argsort(pred)],cmap=cmap,norm=norm)
    plt.axis('off')
    plt.savefig(f'rainfall{k}-{i}.pdf', bbox_inches='tight')
    plt.show()

In [ ]:
for j, pred in enumerate(preds):
    pred_df = pd.DataFrame(data=pred, columns=('group',))
    groups = [
        pred_df.index[pred_df['group'] == i] for i in sorted(set(pred))
    ]

    plt.figure()
    ax = plt.axes()
    ax.set_aspect('equal')

    plot_australia()

    for i, group in enumerate(groups):
        plot_station_group(group, MARKERS[i], COLOURS[i])
    plt.savefig(f'australia{k}-{j}.pdf', bbox_inches='tight')
    plt.show()